In [25]:
import os
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
from models.edsr import edsr
import matplotlib as plt
import seaborn as sns
import math

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [26]:
def quantile_clipping(data, percentage, mode="max"):
    quantile_val = np.quantile(data, percentage)
    if mode == "max":
        data = data.clip(max=quantile_val)
    if mode == "min":
        data = data.clip(min=quantile_val)
    return data

def exp_root_norm(data, exp=2):
    return data ** (1 / exp)

def minmax_scale(images):
    # Assuming images is a 4D array with shape (N, 32, 32)
    min_val = np.min(images)
    max_val = np.max(images)
    
    scaled_images = (images - min_val) / (max_val - min_val)
    
    return scaled_images

def preprocess(images):
    images = quantile_clipping(images, 0.95, mode="max")
    images = exp_root_norm(images, exp=2)
    images = minmax_scale(images)
    return images

import pickle

def save_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [27]:
def create_client_data(fine_matrices, coarse_matrices, num_clients, scale_factor):
    print(f"Distributing different full-size matrices across {num_clients} clients...")
    
    num_samples = fine_matrices.shape[0]
    samples_per_client = num_samples // num_clients

    fine_size = fine_matrices[0].shape[0]
    coarse_size = coarse_matrices[0].shape[0]

    if coarse_size*scale_factor != fine_size:
        print(f"Size mismatch.. adjusting: {fine_size} -> {coarse_size*scale_factor}")
        fine_size = coarse_size*scale_factor
    
    # Let's modify the fine matrices (we select filter only fine_size * fine_size dimension)
    fine_matrices = [fine[:fine_size, :fine_size] for fine in fine_matrices]
    fine_matrices = np.array(fine_matrices).reshape((-1, fine_size, fine_size, 1))

    client_data = []
    for i in range(num_clients):
        start_idx = i * samples_per_client
        end_idx = start_idx + samples_per_client if i < num_clients - 1 else num_samples
        
        client_fine = fine_matrices[start_idx:end_idx]
        client_coarse = coarse_matrices[start_idx:end_idx]
        
        client_data.append((client_coarse, client_fine))
    
    return client_data

In [28]:
# Main script
dataset = 'geant'
if dataset == 'geant':
    original_size = 22
elif dataset == 'germany':
    original_size = 161

scale_factor = 2
num_clients = 2
path_to_data = 'CNSM/data'
ground_truth = f'{dataset}_original_{original_size}.npy'
NUM_ROUNDS = 20

In [24]:
import time

for scale_factor in [2, 3, 4]:
    for num_clients in range(2, 21):

        # Define the TFF model
        def model_fn():
            keras_model = edsr(input_depth=1, scale=scale_factor, num_filters=64, num_res_blocks=8)
            return tff.learning.models.from_keras_model(
                keras_model,
                input_spec=(
                    tf.TensorSpec(shape=(None, coarse_matrix_size, coarse_matrix_size, 1), dtype=tf.float32),
                    tf.TensorSpec(shape=(None, fine_matrix_size, fine_matrix_size, 1), dtype=tf.float32)
                ),
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.MeanSquaredError()]
            )

        def create_tf_dataset_for_client(client_data):
            def batch_format_fn(x, y):
                return (x, y)
            dataset = tf.data.Dataset.from_tensor_slices(client_data)
            dataset = dataset.shuffle(buffer_size=len(client_data[0]))
            dataset = dataset.batch(32)
            dataset = dataset.map(batch_format_fn)
            return dataset

        # Load and preprocess your data
        train_file = f'{dataset}_coarse_{original_size//scale_factor}_x{scale_factor}.npy'
        train_set = np.load(os.path.join(path_to_data, train_file)).astype(np.float32)
        train_ground_truth = np.load(os.path.join(path_to_data, ground_truth)).astype(np.float32)

        train_set = train_set.reshape((-1, original_size//scale_factor, original_size//scale_factor, 1))
        train_ground_truth = train_ground_truth.reshape((-1, original_size, original_size, 1))

        print(f"Train set shape: {train_set.shape}", "Ground truth shape:", train_ground_truth.shape)

        train_set = preprocess(train_set)
        train_ground_truth = preprocess(train_ground_truth)

        # Create client data
        client_data = create_client_data(train_ground_truth, train_set, num_clients, scale_factor)

        # Print the shape of the client data
        for i, client in enumerate(client_data):
            print(f"Client {i} LR:", client[0].shape, "Client {i} HR:", client[1].shape)

        # Shapes of the client data
        coarse_matrix_size = client_data[0][0].shape[1]
        fine_matrix_size = client_data[0][1].shape[1]

        # Convert all clients data to float 32
        client_data = [(x.astype(np.float32), y.astype(np.float32)) for x, y in client_data]
        federated_train_data = [create_tf_dataset_for_client(cd) for cd in client_data]

        # Create the federated learning process
        iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
            model_fn,
            client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=1e-3),
            server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=1e-3)
        )

        # Initialize the server state
        state = iterative_process.initialize()

        # Perform federated training
        NUM_ROUNDS = 20
        start = time.time()
        for round_num in range(NUM_ROUNDS):
            state, metrics = iterative_process.next(state, federated_train_data)
            print(f'Round {round_num}')
            print(metrics)

        end = time.time()
        print(f"Training time: {end - start}")
        # Save training time
        save_pickle(end - start, f'CNSM/fed_data/{dataset}_fed_training_time_{num_clients}_x{scale_factor}_rounds_{NUM_ROUNDS}.pkl')
        # Save the federated weights
        fed_weights = state[0].trainable
        # Save weights
        save_pickle(fed_weights, f'CNSM/fed_data/{dataset}_fed_weights_{num_clients}_x{scale_factor}_rounds_{NUM_ROUNDS}.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'CNSM/data/geant_coarse_11_x2.npy'